In [7]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import pandas as pd

In [8]:
from utils import get_all_top_30
from utils import get_keywords
from utils import get_search_news
from database import WRITEMEDataBase

In [9]:
db = WRITEMEDataBase()

In [10]:
now = datetime.now()
date = '%04d%02d%02d' % (now.year, now.month, now.day)
time = now.strftime("%p").lower()

# 1. News Crawling

In [5]:
df = get_all_top_30()
df.groupby('category').head(1)

,title,text,url,category,date
0,[속보]형제복지원 피해자 농성 중단…여야 '과거사법' 처리 합의,부산 형제복지원 피해자 최승우씨가 7일 농성을 중단하고 내려와 김무성 ...,https://news.naver.com/main/ranking/read.nhn?r...,정치,20200507
30,"[속보]준법감시위, 이재용 대국민 사과에 ”구체적 실행방안 마련해달라“",삼성그룹 준법감시위원회가 7일 이재용 삼성전자 부회장(52)의 대...,https://news.naver.com/main/ranking/read.nhn?r...,경제,20200507
60,"[속보] 교육부 “대구 등교 일정, 날짜 당기는 것만 금지”",[서울신문]개학 앞두고 ‘에어컨 점검 및 교실 환기’ - 고등학교...,https://news.naver.com/main/ranking/read.nhn?r...,사회,20200507
90,"아픈 어머니를 땅속에 파묻은 아들, 어머니는 사흘을 견뎠다",중국 산시성에서 반신불수 노모 생매장며느리 신고로 출동한 경찰이 구출노...,https://news.naver.com/main/ranking/read.nhn?r...,세계,20200507
120,[핵잼 사이언스] 고대 오징어가 물고기 잡아먹는 순간…2억년 전 화석 발견,[서울신문 나우뉴스]고대 오징어가 물고기 잡아먹는 순간…2억년 전...,https://news.naver.com/main/ranking/read.nhn?r...,IT/과학,20200507


# 2. Get keywords

In [6]:
keywords = df.groupby('category').apply(get_keywords)

In [7]:
keywords

category
IT/과학     [벨벳, 카카오, 코로나, 전자, 화석]
경제       [남양유업, 아파트, 조사, 기부, 확진]
사회         [클럽, 확진, 방문, 코로나, 등교]
세계         [아베, 코로나, 가스, 총리, 인도]
정치       [기부, 원내대표, 의원, 지원금, 재난]
dtype: object

# 3. To DB - keyword

In [8]:
db.select_keywords()

()

In [9]:
for category, keyword in keywords.iteritems():
    db.insert_keywords(keyword, category, date, time)

In [16]:
result = db.select_keywords()
result = pd.DataFrame(result)
result.iloc[23:24]

,keywordID,categoryID,keywordName,date,time
23,24,1,기부,2020-05-07,pm


In [37]:
# db.delete_keywords()

# 4. Get hints

In [19]:
keywordID = 24

In [20]:
keywordName = db.keywordName(keywordID)
df = get_search_news(keywordName)
hints = get_keywords(df, only_noun=True)

print(keywordName, hints)

기부 ['기부', '지원금', '대통령', '재난', '긴급']


# 5. To DB - hint

In [44]:
db.select_hints()

[{'keywordID': 1, 'hintID': 1, 'hintName': '벨벳'},
 {'keywordID': 1, 'hintID': 2, 'hintName': '뉴시스'},
 {'keywordID': 1, 'hintID': 3, 'hintName': '디자인'},
 {'keywordID': 1, 'hintID': 4, 'hintName': '영상'},
 {'keywordID': 1, 'hintID': 5, 'hintName': '예약'},
 {'keywordID': 2, 'hintID': 6, 'hintName': '환자'},
 {'keywordID': 2, 'hintID': 7, 'hintName': '코로나'},
 {'keywordID': 2, 'hintID': 8, 'hintName': '한국'},
 {'keywordID': 2, 'hintID': 9, 'hintName': '수출'},
 {'keywordID': 2, 'hintID': 10, 'hintName': '수지'},
 {'keywordID': 3, 'hintID': 11, 'hintName': '카카오'},
 {'keywordID': 3, 'hintID': 12, 'hintName': '대비'},
 {'keywordID': 3, 'hintID': 13, 'hintName': '분기'},
 {'keywordID': 3, 'hintID': 14, 'hintName': '매출'},
 {'keywordID': 3, 'hintID': 15, 'hintName': '전년'},
 {'keywordID': 4, 'hintID': 16, 'hintName': '화석'},
 {'keywordID': 4, 'hintID': 17, 'hintName': '사냥'},
 {'keywordID': 4, 'hintID': 18, 'hintName': '생산'},
 {'keywordID': 4, 'hintID': 19, 'hintName': '쿠스'},
 {'keywordID': 4, 'hintID': 20, 'hin

In [22]:
db.insert_hints(keywordID, hints)

In [22]:
result = db.select_hints()
result = pd.DataFrame(result)
result[result['keywordID']==24]

,keywordID,hintID,hintName
115,24,116,기부
116,24,117,지원금
117,24,118,대통령
118,24,119,재난
119,24,120,긴급
240,24,241,기부
241,24,242,지원금
242,24,243,대통령
243,24,244,재난
244,24,245,긴급


In [38]:
# db.delete_hints()

In [39]:
db.close()